*Ex1_Numeric_Character_Formats.sas (Part 1);

* This code example below shows only single (not multiple) CHARACTER values on the left side of the = sign 
  in the PROC FORMAT VALUE statement.

In [ ]:
Title 'Format for character values';
options nocenter nodate nosource;
proc format;
value $regionfmt
    'AFR' = 'Africa'
    'AMR' = 'Americas'
    'EUR' = 'Europe'
    'EMR'  ='Eastern Mediterranean'
    'SEAR' = 'South-East Asia'
    'WPR' = 'Western Pacific';
      run;
proc freq data=sashelp.demographics; 
  tables region; 
  format region $regionfmt.;
run;


Ex1_Numeric_Character_Formats.sas (Part 2);

In the example below, note the ranges on the left side of the = sign in PROC FORMAT VALUE statement with keywordS: LOW, and OTHER.

In [ ]:
Title 'Format for numeric values';
options nocenter nodate nosource;
proc format;
  value numfmt
           Low - <0  = "Nonresponse"
           0="Never"
           1-5 = "Within past 5 years"
           6-High = "More than 5 years ago"
           . ="Missing" ;
   value $charfmt
           Low-<'0'  = "Nonresponse"
          '0' = "Never"
           '1'-'5' = "Within past 5 years"
           '6'-High = "More than 5 years ago" ; 
 run;
data work.have;
input id $ 1 Colonoscopy 3-4 c_Colonoscopy $6-7;
datalines;
A -1 -1 
B  . 
C 3   3
D -9 -9
F 3  3
G 5  5
H 6  6 
I .   
J 7  7
;
proc freq data=work.have;
tables colonoscopy c_colonoscopy /nopercent nocum missing;
Format colonoscopy numfmt. c_colonoscopy $charfmt.;
run;

Ex1_Numeric_Character_Formats.sas (Part 3);

In the code snippet below, the format values are ranges; the special keyword LOW is used 
to define the lowest value. 

Because it is a numeric format, LOW does not format missing values. Note that, for character 
formats, LOW includes missing or blank values.

Important note: Here we are grouping data values using formats in the PROC FREQ step.


In [ ]:
Title 'Grouping data values using formats in the PROC FREQ step';
options nocenter nodate nonumber nosource;
proc format;
     value wtfmt   low - <100 = 'Under 100 lbs'
                 100 - high = '100+ lbs'; 
run;
proc freq data=sashelp.class;
   tables weight;
   format weight wtfmt.;
run;


In [ ]:
*Ex2_Nested_Formats.sas;
options nocenter nodate nosource;
proc format;
value date_grp_fmt
    low-'03jul1995'd = 'Pre July 4th 1995'
    '04jul1995'd-'31jul1995'd = [mmddyy8.]
    '01aug1995'd-high = 'Aug 1-Dec 31, 1995';
value sales_fmt
    low-<5000 = 'Less than $5,000'
    5000-9999 = '$5,000-<$10,000'
    10000-high = [dollar12.2];
  run;
  title 'Nested formats';
  proc freq data=sashelp.mdv;
    tables shipdate sales93;
    format shipdate date_grp_fmt.  sales93 sales_fmt.;
  run;

Ex3_Picture_Statement.sas (Part 1)

Use the PROC FORMAT PICTURE statement to specify a template 
 (up to 40 characters enclosed in quotation marks) for labeling numbers.

There are three types of characters in the template.
* Digit selectors (e.g., 0 through 9)
* Message characters (e.g., M for Million, B for Billion) 
* Directives (special characters e.g.,  %A %B %d %Y - to format date values)


In [1]:
*Ex3_Picture_Statement.sas (Part 1);
Proc format;
 picture week_x 1-52='99'; /*Non-zero digit selector*/
 picture week_y 1-52 ='00';  /*Zero digit selector*/
 run;
 data have;
   input week @@;
     * Create two new variables based on the original variable;
     week_x = week;
     week_y = week;
 datalines; 
 1 3  6 8 9 14 15 34 52
 ;
options nocenter nodate nosource;
title1 'Ex3_Picture_Statement.sas (Part 1)';
title2 'Non-zero digit selectors in the PICTURE format add zeros to the formatted value for WEEK_X as needed';
title3 'Zero digit selectors in the PICTURE format do not add zeros to the formatted value for WEEK_Y';
proc print data=have noobs;
 var week week_x week_y;
 format week_x week_x. week_y week_y.;
run;
title3;

SAS Connection established. Subprocess id is 7376



week,week_x,week_y
1,01,1
3,03,3
6,06,6
8,08,8
9,09,9
14,14,14
15,15,15
34,34,34
52,52,52


Ex3_Picture_Statement.sas (Part 2)
Create picture formats to display:
* "billion" figures in millions (template showing digit selectors)
* "million" figures in thousands (template showing digit selectors)
* "million" figures with template showing with message charactes

Code explanation (PROC FORMAT features):

* Keywords are low-high, representing the range of non-missing values to 
which the format will be applied.

* The MULT= specifies the number to multiply the variable's value 
before it is formatted.

* The Round option with the PICTURE statement rounds the data to the 
nearest integer before formatting.

* The message character (e.g., M) is inserted into the picture
after the numeric digits are formatted.


In [2]:
*Ex3_Picture_Statement.sas (Part 2);
proc format;
picture thou (round)
      low-high='000,000,000' (mult=.001);
picture mil (round)
      low-high='0,000,000,000' (mult=.000001);
picture m (round)
      low-high='0,000.9 M' (mult=.00001);
run;

  data work.Pop2005;
  input name $1-14 pop: comma.;
  pop_x = pop;
  pop_y=pop;
  pop_z=pop;
  datalines;
CHINA              1,323,344,591
INDIA              1,103,370,802
UNITED STATES        298,212,895
INDONESIA            222,781,487
BRAZIL               186,404,913
PAKISTAN             157,935,075
RUSSIA               143,201,572
;
title1 'Ex3_Picture_Statement.sas (Part 2)';
title2 'User-defined format that expresses the numbers in thousands and millions';
proc print data=work.pop2005 noobs split='*';
label pop='Population*Size'
      pop_x= 'Population*Size*(in millions)'
      pop_y= 'Population*Size*(in thousands)'
      pop_z= 'Population*Size*(in M)';
Format pop comma14. pop_x mil. pop_y thou. pop_z m.;
run;


name,PopulationSize,PopulationSize(in millions),PopulationSize(in thousands),PopulationSize(in M)
CHINA,"1,323,344,591","1,323","1,323,345","1,323.3 M"
INDIA,"1,103,370,802","1,103","1,103,371","1,103.4 M"
UNITED STATES,"298,212,895",298,"298,213",298.2 M
INDONESIA,"222,781,487",223,"222,781",222.8 M
BRAZIL,"186,404,913",186,"186,405",186.4 M
PAKISTAN,"157,935,075",158,"157,935",157.9 M
RUSSIA,"143,201,572",143,"143,202",143.2 M


Ex3_Picture_Statement.sas (Part 3)
Code explanation (PROC FORMAT features):

* The Round option with the PICTURE statement rounds the data to the 
  nearest integer before formatting.

* PREFIX= specifies a character prefix for the formatted value.

* Leading 0's as digit selectors mean blanks.

* Nines mean some values.


In [3]:
*Ex3_Picture_Statement.sas (Part 3);
proc format;
  picture test (round)
         low-<0='09.99' (prefix='-')
         0-<10 ='09.99'
        10-<100='99.9'
       100-999 ='999';
run;

DATA temp;
INPUT  Some_value @@;
 datalines;
  457.677 7.219 0.303 -0.027 95.307 752.789 
  ; 
title1 'Ex3_Picture_Statement.sas (Part 3)';
title2 'User-defined format that expresses the decimal values in percentages';

proc print noobs; 
   var Some_value ;
   format Some_value test.;
run;
  

Some_value
458
7.22
0.30
-0.03
95.3
753


In [4]:
*Ex3_picture_statement.sas (Part 4);
PROC FORMAT;
 PICTURE p_fmt (ROUND)
     LOW-<0 = "009.99%" (PREFIX="-" MULT=10000)
         0-HIGH = "009.99%" (MULT=10000);
PICTURE p_fmt_x (ROUND)
     LOW-<0 = "009.99" (PREFIX="-" MULT=10000)
         0-HIGH = "009.99" (MULT=10000);
RUN;
DATA work.have;
 INPUT Value1 @@;
 Value2 = Value1; Value3 = Value1;
 Value4 = Value1; Value5 = Value1;
DATALINES;
0.0345678  -0.00123456  -0.456789 .120
;
options nodate nonumber;
title1 'Ex3_Picture_Statement.sas (Part 4)';
title2 'SAS Formats and User-Defined Formats Applied';

PROC PRINT DATA=work.have SPLIT="*" NOOBS;
 VAR Value: ;
 FORMAT Value2 PERCENT8.2  Value3 PERCENTN8.2  
        Value4 p_fmt. Value5 p_fmt_x.;
 LABEL Value1="No"*"Format"*"Applied"
       Value2="SAS"*"Percent"*"Format"*"PERCENT8.2"*"Applied"
       Value3="SAS"*"Percent"*"Format"*"PERCENTN8.2"*"Applied"
       Value4="User"*"Picture"*"Format 1"*"Applied"
       Value5="User"*"Picture"*"Format 2"*"Applied";
RUN;


NoFormatApplied,SASPercentFormatPERCENT8.2Applied,SASPercentFormatPERCENTN8.2Applied,UserPictureFormat 1Applied,UserPictureFormat 2Applied
0.03457,3.46%,3.46%,3.46%,3.46
-0.00123,( 0.12%),-0.12%,-0.12%,-0.12
-0.45679,(45.68%),-45.68%,-45.68%,-45.68
0.12000,12.00%,12.00%,12.00%,12.00


Ex3_picture_statement.sas (Part 5)

   The % followed by a letter indicates a directive.
   
   * %A - full weekday name
   * %B - full month name
   * %D - day of the month 
   * %Y - four-digit year
   
   The Datatype PICTURE option specifies that the above format 
   will be applied to a SAS date, SAS time or SAS datetime. 


In [5]:
*Ex3_picture_statement.sas (Part 5);
proc format; 
      picture date_fmt(default = 45)
      other='%A, %B %D,%Y' (datatype=date); 
   run;
 
 data have; 
    some_date1=today(); 
    some_date2='01Jul2019'd;
  run;
options nodate nonumber;
title1 'Ex3_Picture_Statement.sas (Part 5)';
title2 'User-Defined Picture Formats Applied';

PROC PRINT DATA=have NOOBS;
 VAR Some: ; 
format Some:  date_fmt.;
run;


some_date1,some_date2
"Tuesday, July 30,2019","Monday, July 01,2019"


CNTLIN= Option

Creating a User-Defined Format from a SAS Data Set

If there is a long list of variable values and if the values and 
their labels are available in an electronic file 
(ASCII, EXCEL or data base mode), the file can be read into SAS to 
create a SAS data set. There is no need to type this long list under the PROC FORMAT VALUE statement!  

Requirements: The data set must have three required columns–
FMTNAME, START, and LABEL.  The data set can have the optional 
column called the TYPE column with values of ‘C’ for the character variable 
and ‘N’ for the numeric variable.

The CNTLIN=input-control-SAS-data-set (as shown in line 42 below) 
specifies a SAS data set from which PROC FORMAT builds INFORMATs. 
Note that CNTLIN= builds FORMATS and INFORMATS without using a VALUE, 
PICTURE, or INVALUE statement. 


In [6]:
*Ex4_Value_cntlin_compared.sas (Part 1);
proc format ;
value $xcausesfmt A00 = "Cholera "
                  A00.0 = "Cholera due to Vibrio cholerae 01, biovar cholerae" 
                  A00.1= "Cholera due to Vibrio cholerae 01, biovar eltor" 
                  A01.1= "Paratyphoid fever A" 
                  A01.2= "Paratyphoid fever B" 
                  A01.3= "Paratyphoid fever C" 
                  A01.4= "Paratyphoid fever, unspecified" 
                  A02= "Other salmonella infections" 
                  A02.0= "Salmonella enteritis" 
                  A02.1= "Salmonella septicaemia" ;
data have1; 
input id $ cause_dth_code $ @@;
format cause_dth_code $xcausesfmt.; 
datalines; 
12345 A01.4 23456 A01.3 34567 A02.0
; 
title1 "Format created using the PROC FORMAT VALUE statement";
proc print data=have1 noobs; run;

id,cause_dth_code
12345,"Paratyphoid fever, unspecified"
23456,Paratyphoid fever C
34567,Salmonella enteritis


In [9]:
*Ex4_Value_cntlin_compared.sas (Part 2);
data causes_of_death;
 retain FMTNAME '$causesfmt' type 'C';;
input START $ LABEL & $50.;
datalines;
A00    Cholera 
A00.0  Cholera due to Vibrio cholerae 01, biovar cholerae 
A00.1  Cholera due to Vibrio cholerae 01, biovar eltor 
A01.1  Paratyphoid fever A 
A01.2  Paratyphoid fever B 
A01.3  Paratyphoid fever C 
A01.4  Paratyphoid fever, unspecified 
A02    Other salmonella infections 
A02.0  Salmonella enteritis 
A02.1  Salmonella septicaemia
;

proc sort data=causes_of_death
  out=causes_of_death nodupkey;
  by START;
run;
proc format cntlin=causes_of_death;
run;

data have2; 
input id $ cause_dth_code $ @@;
format cause_dth_code $causesfmt.; 
datalines; 
12345 A01.4 23456 A01.3 34567 A02.0
; 
title1 "Format created using the PROC FORMAT cntlin= optiion";
proc print data=have2; run;

Obs,id,cause_dth_code
1,12345,"Paratyphoid fever, unspecified"
2,23456,Paratyphoid fever C
3,34567,Salmonella enteritis


In [12]:
*Ex5_print_catalog_fmtlib.sas;
options nocenter nonumber nodate nosource;
options FORMCHAR = '1----1+1---+=1-1\<>*';
libname sds 'C:\SASCourse\Week3';
proc catalog catalog = sds.catalogpop;
contents;
run;

proc format library = sds.catalogpop fmtlib;
select REGIONFMT;
title 'Formats for Pop2013x Data Set';
run;

Ex6_Finding_Invalids.sas (Part 1)

Method 1

In the INPUT statement below, the ?? format modifier for the 
S_DATE variable suppresses the invalid data message and, 
in addition, prevents the automatic variable _ERROR_ 
from being set to 1 when invalid data are read.

[See SAS® Documentation for details]
Also see Week2 "Ex11_Question_marks.sas


In [13]:
*Ex6_Finding_Invalids.sas (Part 1);
options nodate nonumber;
PROC FORMAT;
VALUE date_fmt 
   LOW-HIGH = 'valid date'
    other='invalid date';  
run;
DATA work.HAVE;
infile datalines firstobs=2;
input Name $ 1-7 
      @8 s_date ?? mmddyy10.
      @8 s_date_ch $10.;
format s_date mmddyy10. ;
datalines;
12345678901234567890
Alfred 04/22/2005
Alice  01/15/2005
Barbara12/20/2004
Carol  10/29/1999
Henry  02/31/2007
Philip 02/31/2005
Ronald 02/29/2006
;
 Title1 'Table lookup using a user-defined format';
proc freq DATA=work.Have;
  table s_date /missing ;
  format S_date date_fmt.;
RUN;

Title1 "Listing of S_DATE_CH values (invalid dates)"; 
proc print DATA=HAVE noobs;
  var Name s_date_ch;
  where S_date= .;
RUN;


Ex6_Finding_Invalids.sas (Part 2)

Method 2
Use a sum statement to accumulate the count of bad dates (i.e., S_date=.) 
during DATA step execution.

During the first iteration, specify the column position as well as the text
to the LOG window.

Output to the LOG window the values of bad dates from the 
   "character date variable" (i.e., S_date_ch) at the specified
   column position if the "numeric date variable" (i.e. S_date) 
   has a missing value.

The format argument is the equal sign so that the text (within quotes)
   as well as the variable name precedes it value.



In [17]:
*Ex6_Finding_Invalids.sas (Part 2);
options nodate nonotes nosource nonumber;
DATA _NULL_;
infile 'C:/SASCourse/Week3/Sample_dates.txt' firstobs=2 END=lastobs;
input Name $ 1-7 
      @8 s_date ?? mmddyy10.
      @8 s_date_ch $10.;

if s_date = . then invalid_dates+1; 
if _n_=1 then put @10 'List of invalid dates:'; 
if s_date = . then put @10 s_date_ch ;
if lastobs then  put @10 'Number of invalid dates = ' invalid_dates;
run;


In [18]:
*Ex8_Invalue_statement.sas (Part 1);
options nocenter nonumber nodate nosource;
proc format ;
    invalue scorefmt (upcase just)
            'A'=95  'B'=84  
            'C'=79  'D'=60;
run;
data Grade_data1;
  input @1 id $4.  @6 grade scorefmt2. ;
datalines;
S001 A 
S002 D 
S003 B 
S004 B
S005  D 
S006 C 
S007 c 
;
title1 'PROC FORMAT INVALUE Invalue Statement with UPCASE and JUST Options (Part 1)';
proc print data=Grade_data1 noobs ; run;

id,grade
S001,95
S002,60
S003,84
S004,84
S005,60
S006,79
S007,79


In [19]:
*Ex8_Invalue_statement.sas (Part 2);
options nocenter nonumber nodate nosource;
proc format ;
    invalue gpafmt (upcase)
            2.5-4.0 = _SAME_
            'B' = 3.0
            OTHER=. 
;
data GPA_data;
  input id $  GPA :gpafmt3. @@;
datalines;
S001 2.8 S002 3.7 S003 4.0 S004 B 
S005 2.7 S006 3.2 S007 . 
;
title1 'PROC FORMAT INVALUE Statement with the UPCASE Option';
title2 'and _SAME_ and OTHER Keywords (Part 2)';
proc print data=GPA_data noobs ;
run;

id,GPA
S001,2.8
S002,3.7
S003,4.0
S004,3.0
S005,2.7
S006,3.2
S007,.


In [20]:
*Ex8_Invalue_statement.sas (Part 3);
options nocenter nonumber nodate nosource;
proc format library=work;                                    
    value respfmt_1x                            
    0="TOTAL"                                              
    1="YES"                                                
    2="NO"   
    -8="DON'T KNOW" 
     -7="REFUSED"
    -9="NOT ASCERTAINED"                                        
    -1="INAPPLICABLE" ;

  value respfmt_2x                            
    0="TOTAL"                                              
    1="YES"                                                
    2="NO"   
    LOW- <0="INVALID"                   
    ;
  run;  
data Fictitious_Data; 
   infile datalines firstobs=2; 
   input @1 Did_you_ever_use_SAS 2. @5 freq 9.;                                                    
datalines;  
0123456789 
-9     251
-8     722
-7       6
-1   28323
 1    4134 
 2    3504 
 0   36940
;       
title1 'PROC FORMAT VALUE Statement (Part 3)'; 
title2;
 proc print data=Fictitious_Data noobs; 
 format Did_you_ever_use_SAS respfmt_1x. freq comma6.;
run;                                              



Did_you_ever_use_SAS,freq
NOT ASCERTAINED,251
DON'T KNOW,722
REFUSED,6
INAPPLICABLE,"28,323"
YES,"4,134"
NO,"3,504"
TOTAL,"36,940"


In [21]:
*Ex9_Create_vars_Different_Ways.sas (part 1);
data Have1;
 length age_group $11;
 input age @@ ;
datalines;
  0 5 10 17 40 48 50 59 62 81 99 100
  ; 
title1 'Frquency Table for a Variable with Discrete (Ungrouped) Data Values (Part 1)';
title2;
proc freq data=Have1;   
 table age; 
run;

age,Frequency,Percent,CumulativeFrequency,CumulativePercent
0,1,8.33,1,8.33
5,1,8.33,2,16.67
10,1,8.33,3,25.00
17,1,8.33,4,33.33
40,1,8.33,5,41.67
48,1,8.33,6,50.00
50,1,8.33,7,58.33
59,1,8.33,8,66.67
62,1,8.33,9,75.00
81,1,8.33,10,83.33


In [22]:
*Ex9_Create_vars_Different_Ways.sas (part 2);
proc format ;
 value agefmt low-17 = '0-17 Years'
              18-49 = '18-49 Years'
              50-64 = '50-64 Years'
              65-High = '65+ Years' ;
data Have2;
 input age @@ ;
 datalines;
  0 5 10 17 40 48 50 59 62 81 99 100
  ; 
title1 'Frequency Table by Grouping Data Values Using Formats in PROC FREQ (Part 2)';
title2;
proc freq data=Have2; 
 table age; format age agefmt.;
run;


age,Frequency,Percent,CumulativeFrequency,CumulativePercent
0-17 Years,4,33.33,4,33.33
18-49 Years,2,16.67,6,50.00
50-64 Years,3,25.00,9,75.00
65+ Years,3,25.00,12,100.00


In [23]:
*Ex9_Create_vars_Different_Ways.sas (part 3);
 data Have3;
 length age_group $11;
  input age @@ ; 

  /*Character-type categorical variables using an assignment statement;
  IF-THEN/Else Statements are used to conditionally assign values to variables.*/

  if age <=17 then age_group='0-17 Years';
  else if 18<=age<=49 then age_group= '18-49 Years';
  else if 50<=age<=64 then age_group= '50-64 Years';
  else if age>=65 then age_group= '65+ Years';
  datalines;
  0 5 10 17 40 48 50 59 62 81 99 100
  ; 
 options center;
 title1 'Frequency Table for a Variable Created with an Assignment Statement';
 title2 '(IF-THEN/ELSE-IF-THEN in DATA Step - Part 3)';

proc freq data=Have3; 
 table age_group ; 
run;


age_group,Frequency,Percent,CumulativeFrequency,CumulativePercent
0-17 Years,4,33.33,4,33.33
18-49 Years,2,16.67,6,50.00
50-64 Years,3,25.00,9,75.00
65+ Years,3,25.00,12,100.00


In [24]:
 *Ex9_Create_vars_Different_Ways.sas (part 4);
 proc format ;
 value agefmt low-17 = '0-17 Years'
              18-49 = '18-49 Years'
              50-64 = '50-64 Years'
              65-High = '65+ Years' ;
  data Have4;
 input age @@ ; 
  *Character-type categorical variables using a PUT function;
  age_group_x=put(age, agefmt.);
  datalines;
  0 5 10 17 40 48 50 59 62 81 99 100
  ; 
 title1 'Frequency Table for a Variable Created with an Assignment Statement';
 title2 'and the PUT Function in DATA Step (Part 4)';
 proc freq data=Have4; 
 table age_group_x ; 
run;


age_group_x,Frequency,Percent,CumulativeFrequency,CumulativePercent
0-17 Years,4,33.33,4,33.33
18-49 Years,2,16.67,6,50.00
50-64 Years,3,25.00,9,75.00
65+ Years,3,25.00,12,100.00


In [25]:
*Ex10_DO_Group.sas;
 data age_data;
  length age_group $14 xage_group $25;
  input age @@ ; 
   if age <=17 then 
       DO;
        age_group ='<=17 Years';
        xage_group= 'Infants-Young Adolesents';
       END;
    else if 18<=age<=34 then 
       DO;
         age_group= '18-34 Years';
         xage_group= 'Young Adults'; 
       END;

    else if 35<=age<=54 then 
       DO;
         age_group= '35-54 Years';
         xage_group='Middle Aged Adults';
       END;
    else if age>=55 then 
      DO;
        age_group= '55+ Years';
        xage_group= 'Older Adults';
      END; 
   datalines;
  0 5 10 17 40 48 50 59 62 81 99 100
  ; 
title1 'Listing of Multiple Variables Created with DO Group';
title2;
 proc print noobs; var age age_group xage_group; 
run;


age,age_group,xage_group
0,<=17 Years,Infants-Young Adolesents
5,<=17 Years,Infants-Young Adolesents
10,<=17 Years,Infants-Young Adolesents
17,<=17 Years,Infants-Young Adolesents
40,35-54 Years,Middle Aged Adults
48,35-54 Years,Middle Aged Adults
50,35-54 Years,Middle Aged Adults
59,55+ Years,Older Adults
62,55+ Years,Older Adults
81,55+ Years,Older Adults


In [27]:
*Ex11_Format_Put_Function.sas;
options nocenter nonumber nodate;
title1; title2;
PROC FORMAT; 
     value regionfmt
        1='Northeast' 2='Midwest' 
        3='South'  4='West';
run;
LIBNAME SDS "C:\SASCourse\Week3";
data Have;
  set sds.pop2013x;
  region_char=put(region, regionfmt.);
run;
proc contents data=Have 
     (keep=region region_char) varnum;
ods select position;
run;
proc print data=Have;
  var region region_char st_name;
  where st_name like 'A%';
run;
title1; title2;


In [28]:
*Ex12_IFC_IFN_Function.sas;
proc format;
    value agefmt
       . = 'Unknown'
       1 = 'TRUE'
       0 = 'FALSE';
 data IFC1_IFN1;
  length age 3 age_group_IFC1 $10;
  input age @@ ; 
    age_group_IFC1 = IFC(0<=age<=64, '0-64 Years', '65+ Years'); 
    age_LE64_IFN1 = IFN(0<=age<=64, 1, 0);
    age_LE64_IFN1_formatted = put(age_LE64_IFN1, agefmt.);
  drop age_LE64_IFN1;
  datalines;
  0 5 10 17 40 48 50 59 62 81 99 100
  ; 
 title1 'Ex12_IFC_IFN_Function.sas';
 proc print noobs; run;
 title1;

 

age,age_group_IFC1,age_LE64_IFN1_formatted
0,0-64 Years,TRUE
5,0-64 Years,TRUE
10,0-64 Years,TRUE
17,0-64 Years,TRUE
40,0-64 Years,TRUE
48,0-64 Years,TRUE
50,0-64 Years,TRUE
59,0-64 Years,TRUE
62,0-64 Years,TRUE
81,65+ Years,FALSE


In [29]:
*Ex13_IFN_Fourth_Argument.sas;
DATA Work.Ifn_Func;
INPUT property_value;
property_tax = ifn(property_value GE 150000,
               property_value*.02,
             property_value*.015, .);
format property_value property_tax dollar8.;
datalines;
150000
. 
250000
100000
;
title1 'Ex13_IFN_Fourth_Argument.sas';
proc print data=Work.Ifn_Func; 
run;
title1;

Obs,property_value,property_tax
1,"$150,000","$3,000"
2,.,.
3,"$250,000","$5,000"
4,"$100,000","$1,500"


In [30]:
*Ex14_in_whichn_whichc.sas;
* Adapted from Joe Matise's code  (SAS-L) - 5/16/2016;
 data have1;
    set sashelp.class (obs=3);
    Age_13_dummy = whichn(13,age);
    Gender_male_dummy= whichc('M',sex);
  run;
 title1 'Ex14_in_whichn_whichc.sas';
 proc print data=have1 noobs;  run;
 title1;


Name,Sex,Age,Height,Weight,Age_13_dummy,Gender_male_dummy
Alfred,M,14,69.0,112.5,0,1
Alice,F,13,56.5,84.0,1,0
Barbara,F,13,65.3,98.0,1,0


In [31]:
*Ex15_input_function.sas (Part 1);
options nocenter nonumber nodate nosource;
data input_function_data;
  c_id = '12345678'; 
  n_id =input(c_id, 8.); 
  c_id_x =input(c_id, $8.);
  c_amount = '$23,500'; 
  n_amount = input(c_amount, dollar7.);
  c_amount_x = '9999';
  n_amount_x = input(c_amount_x, 5.2);
  c_date = '13Sep1963'; 
  n_date = input(c_date, date9.);
FORMAT n_date date9.  n_amount dollar7.; 
putlog (_ALL_) ( =/ +2);  
run;

In [32]:
*Ex15_input_function.sas (Part 2);
title1 'Ex15_input_function.sas (Part 2)' ;
proc contents data=input_function_data varnum; 
ods select position;
run;
title1;

In [33]:
*Ex15_input_function.sas (Part 3);
** ?? Modifier used in the INPUT function;
data _null_;
input c_date $; 
n_date=input(c_date, ?? mmddyy10.);
put c_date= n_date=date9.;
datalines;
02292017
03202017
;


In [34]:
*Ex16_put_function.sas;
proc format;
 value $stypeF  U='Undergraduate'  G='Graduate' ;
run;  
data put_function_data;
c_stype1 ='U';  
c_stype2 = put(c_stype1, $stypeF.);
n_id = 12345678; 
c_id =put(n_id, 8.);
n_amount = 23500; 
c_amount = put(n_amount, dollar7.);
n_date = 1357;    
c_date = put(1357, Weekdate.);
FORMAT n_date Weekdate.  n_amount dollar7.; 
putlog (_ALL_) (=/ +2);
run;
*Ex16_put_function.sas (Part 2);
title1 'Ex16_put_function.sas (Part 2)';
proc contents data=put_function_data varnum; 
ods select position;
run;
title1;


In [37]:
*Ex17_Temporary_Permanent_Catalogs.sas;
options notes source;
PROC FORMAT; 
     value regionfmt
        1='Northeast' 2='Midwest' 
        3='South'  4='West';
run;
LIBNAME library 'C:\SASCourse\Week3';
PROC FORMAT LIBRARY=library; 
     value regionfmt
        1='Northeast' 2='Midwest' 
        3='South'  4='West';
run;
LIBNAME sds 'C:\SASCourse\Week3';
PROC FORMAT LIBRARY=sds; 
     value x_regionfmt
       1='Northeast' 2='Midwest'
       3='South' 4='West';
run;
LIBNAME xsds 'C:\SASCourse\Week3';
PROC FORMAT LIBRARY=xsds.catalogpop; 
     value regionfmt
        1='Northeast' 2='Midwest' 
        3='South'  4='West';
run;

In [38]:
*Ex18_LIBRARY_library.sas;
OPTIONS nodate nonumber nocenter;
%LET path=C:\SASCourse\Week3;
LIBNAME sds "&path";
LIBNAME library "&path";
title1 'Ex18_LIBRARY_library.sas';
PROC FREQ data=sds.pop2013x; 
 TABLES REGION;
RUN;
title1;


In [39]:
*Ex18_LIBRARY_library.sas;
OPTIONS nodate nonumber nocenter;
%LET path=C:\SASCourse\Week3;
LIBNAME sds "&path";
LIBNAME library "&path";
title1 'Ex18_LIBRARY_library.sas';
PROC FREQ data=sds.pop2013x; 
 TABLES REGION;
RUN;
title1;


Ex20_SELECT_WHEN.sas

[See example code on SELECT WHEN here](https://documentation.sas.com/?docsetId=ds2ref&docsetTarget=n01kskkbawu6isn1vee8uw50jzvu.htm&docsetVersion=9.4&locale=en)

### Using WHEN Statements in a SELECT Group

* Evaluating the when-expression When a select-expression Is Included
* Evaluating the when-expression When a select-expression Is Not Included
* Evaluating the when-expression When a statement-list Is Not Included
Comparisons

### Examples

* Example 1: SELECT with a select-expression
* Example 2: SELECT without a select-expression
* Example 3: SELECT without an IF Expression

In [40]:
*Ex21_create_variable_in_SQL.sas (Part 1);
options nocenter nodate nonumber;
title1 'Ex21_create_variable_in_SQL.sas (Part 1)';
PROC SQL;
SELECT 
  CASE
  WHEN weight <100 THEN '<100 lbs'
  WHEN weight GE 100 AND weight LT 120 THEN '100-<120 lbs'
  WHEN weight GE 120 AND weight LE 150 THEN '120-150 lbs'
  ELSE '120-150 lbs'
  END AS Weight_Cat label= 'Weight Category',
  count(*) as freq_count
FROM sashelp.class
group by Weight_Cat
order by  freq_count desc;
quit;

Weight Category,freq_count
<100 lbs,10
100-<120 lbs,6
120-150 lbs,3


In [41]:
*Ex22_Retain_Sum_Statement.sas (Part 1);
DATA temp ;
  INPUT month sales @@;
      Total_sales+sales;
 FORMAT Sales Total_sales dollar8.;
  DATALINES;
    1 4000 2 5000 3 . 4 5500 5 5000 
    ;
title1 'Ex22_Retain_Sum_Statement.sas (Part 1)';
title2 'SUM Statement';
PROC PRINT noobs; run;

month,sales,Total_sales
1,"$4,000","$4,000"
2,"$5,000","$9,000"
3,.,"$9,000"
4,"$5,500","$14,500"
5,"$5,000","$19,500"


In [42]:
*Ex22_Retain_Sum_Statement.sas (Part 2);
DATA temp1 ;
   RETAIN Total_sales 0;
   FORMAT Sales Total_sales dollar8.;
   INPUT month sales @@;
    Total_sales= sum(Total_sales, sales);
   DATALINES;
   1 4000 2 5000 3 . 4 5500 5 5000 
   ;
title1 'Ex22_Retain_Sum_Statement.sas (Part 2)';
title2 'RETAIN Statement';
PROC PRINT data=temp1; 
  VAR month sales Total_sales;
run;

Obs,month,Sales,Total_sales
1,1,"$4,000","$4,000"
2,2,"$5,000","$9,000"
3,3,.,"$9,000"
4,4,"$5,500","$14,500"
5,5,"$5,000","$19,500"


Overriding the Default Behavior of the Sum Variable

By default, the sum variable is automatically set to 0
before the first observation is read. 

To reset the sum variable to a different number, 
you need to use the RETAIN statement.  

In [43]:
*Ex22_Retain_Sum_Statement.sas (Part 3);
DATA temp;
   RETAIN Total_sales 1000;
   INPUT month sales @@;
    Total_sales+sales;
   FORMAT Sales Total_sales dollar8.;
   DATALINES;
   1 4000 2 5000 3 . 4 5500 5 5000 
   ;
title1 'Ex22_Retain_Sum_Statement.sas (Part 3)';
title2 'RETAIN and SUM Statements';
PROC PRINT noobs; 
 var month sales Total_sales;
RUN;


month,sales,Total_sales
1,"$4,000","$5,000"
2,"$5,000","$10,000"
3,.,"$10,000"
4,"$5,500","$15,500"
5,"$5,000","$20,500"


In [44]:
*Ex22_Retain_Sum_Statement.sas  (Part 4);
proc sort data = sashelp.cars out=cars; by make; run;
data cars_x;
  set cars;
  count + 1;
  by make;
  if first.make then count = 1;
  if last.make;
run;
title1 'Ex22_Retain_Sum_Statement.sas (Part 4)';
title2 'SUM Statement';
proc print data=cars_x;
var make count;
run;
title1; title2;



Obs,Make,count
1,Acura,7
2,Audi,19
3,BMW,20
4,Buick,9
5,Cadillac,8
6,Chevrolet,27
7,Chrysler,15
8,Dodge,13
9,Ford,23
10,GMC,8


In [45]:
*Ex23_read_data_add_format.sas ;
OPTIONS nocenter nodate nonumber;
%LET Path=C:\SASCourse\Week3;
FILENAME raw "&Path\pop2013_no_headers.txt";
LIBNAME SDS "&Path";
PROC FORMAT; 
     value regionfmt
       1='Northeast' 2='Midwest'
       3='South' 4='West';
run;
DATA SDS.Pop;
 INFILE raw DLM=',';
 input sumlev region division fips name :$22.
         pop :comma12.
         pop18p :comma12.  p_pop18p; 
  LABEL region ='Region'     
        FIPS ='State FIPS'
        name ='State Name'  
        pop ='Population*(All Ages)'
        pop18p ='Population*(Aged 18+)'
        p_pop18p = 'Percent*Population*(Aged 18+)'; 
  FORMAT pop pop18p comma12. FIPS z2.;
run;
proc sort data=SDS.pop; by descending pop; run;
title1 'Ex23_read_data_add_format.sas';
title2 'U.S. Population, 2013 (Seven largest states)';
proc print data=SDS.pop (obs=8) split='*' noobs;
var name pop pop18p;
run;
title1; title2;
  


State Name,Population(All Ages),Population(Aged 18+)
California,"38,332,521","29,157,644"
Texas,"26,448,193","19,406,207"
New York,"19,651,127","15,411,151"
Florida,"19,552,860","15,526,186"
Illinois,"12,882,135","9,858,828"
Pennsylvania,"12,773,801","10,058,156"
Ohio,"11,570,808","8,920,978"
Georgia,"9,992,167","7,502,458"


In [46]:
options nocenter nodate nonumber nosource;
title1 'Ex24_remove_labels_formats_informats.sas (Part 1)';
title2 'Metadata in sashelp.mdv';
proc contents data=sashelp.mdv varnum;
ods select position;
run;

data mdv;
  set sashelp.mdv;
 run;
title1 'Ex24_remove_labels_formats_informats.sas (Part 2)';
title2 'After label, format, and informat removed from sashelp.mdv';
title3 'Metata data in new dataset called MDV';

proc datasets lib=work memtype=data nolist;
     modify mdv;
     attrib _all_ label=' ';
     attrib _all_ format=;
     attrib _all_ informat=;
run;
quit;
proc contents data=mdv varnum;
ods select position;
run;

title1; title2; title3;


Ex25_date_group.sas (Part 1)

Simulate male and female expense data for a total of 200 days
  between Jan 1-Dec 31, 2016
 

In [47]:
data work.Have (drop=rownum date_a date_b);
call streaminit(123);
  date_a="01JAN2016"d;
  date_b="31DEC2016"d;
  
  do rownum = 1 to 100;
    do gender = 1 to 2; 
   	Date_p=date_a + floorz((date_b-date_a) * rand("uniform"));
    expense = floorz(1000*rand("Uniform"));
    format date_p date9.;
    output;
  end;
 end;
run;

Adapted from Pete Lund (2011) 

 Create format for overlapping date ranges  (e.g., month-to-date, quarter-to-date, and year-to-date).

  In the VALUE statement with the PROC FORMAT,  the multilabel option maps values to more 
  than one label.

The NOTSORTED option is used on the VALUE statement 
to maintain the desired order.

Calculate expenses for overlapping date ranges.

"The MLF option is used in the reporting procedure to 
allow for processing of formats with multiple labels. 
When we want to use this format in a procedure we need to 
specify the appropriate options for both the 
MULTILABEL and NOTSORTED options." 

Using SAS® Formats:  So Much More than “M” = “Male” 
by Pete Lund - 2011 SAS Global Forum 



In [48]:
*Ex25_date_group.sas (Part 2);
proc format;
value MultiDates (notsorted multilabel)
'1sep2016'd - '30sep2016'd = 'MTD'
'1jul2016'd - '30sep2016'd = 'QTD'  
'1jan2016'd - '31Dec2016'd = 'YTD' 
; 

 value gender_fmt 1 = 'Male'
                  2 = 'Female';
run;
proc tabulate data=work.Have; 
  class date_p gender / mlf preloadfmt order=data;
  var expense;
  format date_p MultiDates.;
  tables (gender all),  date_p*expense*sum*f=dollar10.;
 format gender gender_fmt.;
run; 


In [49]:
*Ex26_delete_rows_all_vars_missing (Part 1);
data have1;
  set sashelp.class(obs=7);
  if mod(_n_,3)=0 then call missing(of _all_);
run;
title1 'Data Set Have1'; 
proc print data=have1; run;


Obs,Name,Sex,Age,Height,Weight
1,Alfred,M,14,69.0,112.5
2,Alice,F,13,56.5,84.0
3,,,.,.,.
4,Carol,F,14,62.8,102.5
5,Henry,M,14,63.5,102.5
6,,,.,.,.
7,Jane,F,12,59.8,84.5


In [50]:
*Ex26_delete_rows_all_vars_missing (Part 2);
data have2;
set have1;
  if cmiss(of _all_) >0 then delete;
run;
title1 'Data Set Have2'; 
proc print data=have2; run;

Obs,Name,Sex,Age,Height,Weight
1,Alfred,M,14,69.0,112.5
2,Alice,F,13,56.5,84.0
3,Carol,F,14,62.8,102.5
4,Henry,M,14,63.5,102.5
5,Jane,F,12,59.8,84.5


In [53]:
*Ex26_delete_rows_all_vars_missing (Part 3);
*Posted by Quentin McMullen to SAS=L 9/25/2018;
data have3;
set have1;
if compress(cats(of _all_),'.')=' ' then delete;
run;
title1 'Data Set Have3'; 
proc print data=have3; run;
title1;

Obs,Name,Sex,Age,Height,Weight
1,Alfred,M,14,69.0,112.5
2,Alice,F,13,56.5,84.0
3,Carol,F,14,62.8,102.5
4,Henry,M,14,63.5,102.5
5,Jane,F,12,59.8,84.5


In [54]:
*Ex28_UserDefinedFormat.sas;
options nocenter nodate nonumber;
/*SAS-L 11/9/2018 by Ian Whitlock */
data class;
  format
      Name    $7.
      Sex     $1.
      Age     3.
      Height  5.1
      Weight  5.1;
  set sashelp.class (obs=7);
run;quit;
proc print data=class noobs; 
run;

Name,Sex,Age,Height,Weight
Alfred,M,14,69.0,112.5
Alice,F,13,56.5,84.0
Barbara,F,13,65.3,98.0
Carol,F,14,62.8,102.5
Henry,M,14,63.5,102.5
James,M,12,57.3,83.0
Jane,F,12,59.8,84.5


In [55]:
* Ex29_Week_3_List_of_Files.sas;
PROC IML;
SUBMIT / R;
setwd ("C:/SASCourse/Week3")
list.files(pattern="SAS", 
           full.names = TRUE, 
           ignore.case = TRUE)
ENDSUBMIT;
QUIT;
